In [2]:
!pip install google
!pip install -U git+https://github.com/adbar/trafilatura.git

import re
import pandas as pd
import numpy as np
import trafilatura
import pprint

     |████████████████████████████████| 45 kB 1.2 MB/s eta 0:00:01
  Cloning https://github.com/adbar/trafilatura.git to /tmp/pip-req-build-bd1d6rpi
  Running command git clone -q https://github.com/adbar/trafilatura.git /tmp/pip-req-build-bd1d6rpi
     |████████████████████████████████| 860 kB 4.7 MB/s eta 0:00:01
  Created wheel for trafilatura: filename=trafilatura-0.4.1-py3-none-any.whl size=154799 sha256=5ef7ce3ca2cbf87e6f554e1626b3e401d9ab991928b00c0f8b07d7e09593c2f4
  Stored in directory: /tmp/pip-ephem-wheel-cache-_xefl_3v/wheels/48/ed/70/138e35a5e7304fa89c2484d3c24a3e0320ac0713670cb03cb3
Successfully built trafilatura


In [3]:
uQuery_1 = "Medigap" # here is where everything begins: we choose two queries that we like to compare
uQuery_2 = "Medicare Supplement Insurance"

uNum = 15

def getResults(uQuery, uTLD, uNum, uStart, uStop):
  try: 
      from googlesearch import search 
  except ImportError:  
      print("No module named 'google' found") 
  
  # What are we searching for 
  query = uQuery
  
  # Prepare the data frame to store urls
  d = []

  for j in search(query, tld=uTLD, num=uNum, start=uStart, stop=uStop, pause=2): 
      d.append(j)
      print(j)
  return d

results_1 = getResults(uQuery_1, "com", uNum, 1,uNum)
results_2 = getResults(uQuery_2, "com", uNum, 1,uNum)

https://www.medicare.gov/medigap-supplemental-insurance-plans/
https://www.investopedia.com/articles/personal-finance/071014/medigap-vs-medicare-advantage-which-better.asp
https://www.investopedia.com/terms/m/medigap-insurance.asp
https://www.cms.gov/Medicare/Health-Plans/Medigap
https://www.medicareresources.org/medicare-benefits/medigap/
https://www.mymedicarematters.org/coverage/medigap/
https://www.aarp.org/health/medicare-qa-tool/what-is-medigap-insurance/
https://www.medicareinteractive.org/get-answers/medicare-health-coverage-options/supplemental-insurance-for-original-medicare-medigaps/medigap-overview
https://www.bcbs.com/medicare/medigap
https://www.medicareinteractive.org/get-answers/medicare-health-coverage-options/supplemental-insurance-for-original-medicare-medigaps/comparing-medigap-options
https://en.wikipedia.org/wiki/Medigap
https://medicareadvocacy.org/medicare-info/medigap/
https://www.thestreet.com/personal-finance/insurance/health-insurance/medigap-vs-medicare-adv

In [6]:
pd.set_option('display.max_colwidth', -1) # make sure output is not truncated (cols width)
pd.set_option("display.max_rows", -1) # make sure output is not truncated (rows)

def readResults(urls, query):
    # Prepare the data frame to store results
    x = []
    position = 0 # position on the serp

    # Loop items in results
    for page in urls:
       position += 1
       downloaded = trafilatura.fetch_url(page)
       if downloaded is not None: # assuming the download was successful
        result = trafilatura.extract(downloaded, include_tables=False, include_formatting=False, include_comments=False) 
        x.append((page, result, query, position))
    return x

d = readResults(results_1, uQuery_1) # get results from there 1st query
e = readResults(results_2, uQuery_2) # get results from there 2nd query

df_1 = pd.DataFrame(d, columns=('url', 'result', 'query', 'position')) # store data in a data frame
df_2 = pd.DataFrame(e, columns=('url', 'result', 'query', 'position')) # store data in a data frame

df_final = pd.concat([df_1, df_2])
print("total number of articles (before filtering) ",len(df_final))

# Remove rows where result is empty 
df_final['result'].replace(' ', np.nan, inplace=True)
df_final = df_final.dropna(subset=['result'])

# Remove rows where article are less than 200 characters in lenght
df_final = df_final[df_final['result'].apply(lambda x: len(str(x))>100)]


# Reindex df
df_final.index = range(len(df_final.index))

# Set the file name
uQuery = uQuery_1 + "_" + uQuery_2
cleanQuery = re.sub('\W+','', uQuery)
file_name = cleanQuery + ".csv"

# Store data to CSV
df_final.to_csv(file_name, encoding='utf-8', index=True)
print("total number of articles saved on",file_name, len(df_final))

total number of articles (before filtering)  28
total number of articles saved on Medigap_MedicareSupplementInsurance.csv 28


In [7]:
# Getting additional hourse power - adding more libraries
!pip install scattertext

%matplotlib inline
import scattertext as st
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS

import io
from scipy.stats import rankdata, hmean, norm
import spacy
import os, pkgutil, json, urllib
from urllib.request import urlopen
from IPython.display import IFrame
from IPython.core.display import display, HTML
from scattertext import CorpusFromPandas, produce_scattertext_explorer
display(HTML("<style>.container { width:98% !important; }</style>"))

nlp = spacy.load('en') # make sure you have the right language here 

     |████████████████████████████████| 6.9 MB 2.2 MB/s eta 0:00:01
     |████████████████████████████████| 8.7 MB 30.8 MB/s eta 0:00:01
     |████████████████████████████████| 231 kB 48.7 MB/s eta 0:00:01


/home/mike/.pyenv/versions/3.8.2/lib/python3.8/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_extraction.stop_words module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_extraction.text. Anything that cannot be imported from sklearn.feature_extraction.text is now part of the private API.
  warnings.warn(message, FutureWarning)


In [8]:
df_final['index'] = df_final.index
df_final.groupby('query').apply(lambda x: x.result.apply(lambda x: len(x.split())).sum())
df_final['parsed'] = df_final.result.apply(nlp) # run NER using spaCy

# Turn it into a Scattertext corpus
corpus = (st.CorpusFromParsedDocuments(df_final, 
                                       category_col='query', 
                                       parsed_col='parsed')
          .build()
          .remove_terms(ENGLISH_STOP_WORDS, ignore_absences=True)) # remove stop words in English  